# Fedbiomed Researcher to train a federated scikit learn model.

## Perceptron
Binary Classification
### Purpose of the exercise :
Three datasets `c1.csv` , `c2.csv` and `c3.csv` has been generated with a target column of 2 different classes.
We will fit a Perceptron (classifier) using Federated Learning.

### Get the data 

We use the make_classification dataset from sklearn datasets

In [ ]:
from sklearn import datasets
import numpy as np

In [ ]:
X,y = datasets.make_classification(n_samples=300, n_features=20,n_clusters_per_class=2, weights=None, flip_y=0.01, class_sep=1.0, hypercube=True, shift=0.0, scale=1.0,shuffle=True, random_state=123)
X.shape,y.shape

In [ ]:
np.unique(y)

#### Creating unbalanced dataset, with different amount of data per centers

In [ ]:
C1 = X[:150,:]
C2 = X[150:250,:]
C3 = X[250:300,:]

y1 = y[:150].reshape([150,1])
y2 = y[150:250].reshape([100,1])
y3 = y[250:300].reshape([50,1])

C1.shape ,C2.shape , C3.shape , y1.shape, y2.shape, y3.shape

In [ ]:
n1 = np.concatenate((C1, y1), axis=1)
np.savetxt('./data/c1.csv',n1,delimiter=',')

n2 = np.concatenate((C2, y2), axis=1)
np.savetxt('./data/c2.csv',n2,delimiter=',')

n3 = np.concatenate((C3, y3), axis=1)
np.savetxt('./data/c3.csv',n3,delimiter=',')

### Setting the node up
Before running this notebook you need to configure 2 nodes: <br/>
* **Node 1 :** `fedbiomed node -p my-first-node dataset add`
  * Select option 1 to add a csv file to the node
  * Choose the name, tags and description of the dataset (you can write 'perp' always and it will be good)
  * Pick the c1.csv file in your machine (in `notebooks/data/c1.csv`) and move it into your node (`my-first-node/data`).
  * Check that your data has been added in node 1 by executing `fedbiomed node -p my-first-node list`
  * Run the node using `fedbiomed node -p my-first-node start`. <br/>

* **Node 2 :** Open a second terminal and run `fedbiomed node -p my-second-node dataset add`
  * Select option 1 to add a csv file to the node
  * Choose the name, tags and description of the dataset (you can write 'perp' always and it will be good)
  * Pick the c2.csv file in your machine (in `notebooks/data/c2.csv`)  and move it into your node (`my-second-node/data`).
  * Check that your data has been added in node 2 by executing `fedbiomed node -p my-second-node list`
  * Run the node using `fedbiomed node -p my-second-node start`.
 

 Wait until you get `Starting task manager`, it means node is online.


**model_args** is a dictionnary containing your model arguments, in case of SGDRegressor this will be max_iter and tol.

**training_args** is a dictionnary with parameters , related to Federated Learning. 

In [ ]:
n_features = 20
n_classes = 2

model_args = {'max_iter':1000, 'tol': 1e-3 , 
               'n_features' : n_features, 'n_classes' : n_classes}

training_args = {   
    'epochs': 5,
    'loader_args': { 'batch_size': 1 }
}

Hereafter the template of the class you should provide to Fedbiomed :
    
**training_data** : you must return here the (X,y) that must be of the same type of 
your method partial_fit parameters. 

In [ ]:
from fedbiomed.common.training_plans import FedPerceptron
from fedbiomed.common.data import DataManager


class PerceptronTraining(FedPerceptron):
    def training_data(self):
        NUMBER_COLS = 20
        dataset = pd.read_csv(self.dataset_path,header=None,delimiter=',')
        X = dataset.iloc[:,0:NUMBER_COLS].values
        y = dataset.iloc[:,NUMBER_COLS]       
        return DataManager(dataset=X,target=y.values, shuffle=True)

In [ ]:
from fedbiomed.researcher.federated_workflows import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['perp']
rounds = 2

# search for corresponding datasets across nodes datasets
exp = Experiment(tags=tags,
                 model_args=model_args,
                 training_plan_class=PerceptronTraining,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)


In [ ]:
exp.run()

Save trained model to file

In [ ]:
exp.training_plan().export_model('./trained_model')

## Lets validate the trained model with the test dataset c3.csv.

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('./data/c3.csv')

In [ ]:
X_test = data.iloc[:,:n_features]
y_test = data.iloc[:,n_features]

F1 score computed with federated algorithm :

For that, we are exporting `exp.aggregated_params()` containing models parameters collected at the end of each round

In [ ]:
from sklearn.metrics import f1_score
loss_metric = f1_score
    
testing_error = []

for i in range(rounds):
    fed_model = exp.training_plan().model()
    fed_model.coef_ = exp.aggregated_params()[i]['params']['coef_']
    fed_model.intercept_ = exp.aggregated_params()[i]['params']['intercept_']
    metric = loss_metric(fed_model.predict(X_test),y_test.ravel())
    print('F1 score metric: ', metric, )
    testing_error.append(metric)

##  SGD regressor

Follwing example uses Adni Dataset please see `README` in the notebooks directory for the insturctions to load Adni dataset into Fed-BioMed nodes. 


In [ ]:
from fedbiomed.researcher.requests import Requests
from fedbiomed.researcher.config import config
req = Requests(config)
req.list(verbose=True)

In [ ]:
from fedbiomed.common.training_plans import FedSGDRegressor
from fedbiomed.common.data import DataManager

class SGDRegressorTrainingPlan(FedSGDRegressor):
    def training_data(self):
        dataset = pd.read_csv(self.dataset_path,delimiter=',')
        regressors_col = ['AGE', 'WholeBrain.bl',
                          'Ventricles.bl', 'Hippocampus.bl', 'MidTemp.bl', 'Entorhinal.bl']
        target_col = ['MMSE.bl']
        
        # mean and standard deviation for normalizing dataset
        # it has been computed over the whole dataset
        scaling_mean = np.array([72.3, 0.7, 0.0, 0.0, 0.0, 0.0])
        scaling_sd = np.array([7.3e+00, 5.0e-02, 1.1e-02, 1.0e-03, 2.0e-03, 1.0e-03])
        
        X = (dataset[regressors_col].values-scaling_mean)/scaling_sd
        y = dataset[target_col]
        return DataManager(dataset=X, target=y.values.ravel(), shuffle=True)
    

In [ ]:
from fedbiomed.common.metrics import MetricTypes
RANDOM_SEED = 1234


model_args = {
    'max_iter':2000,
    'tol': 1e-5,
    'eta0':0.05,
    'n_features': 6,
    'random_state': RANDOM_SEED
}

training_args = {
    'epochs': 5,
    'loader_args': { 'batch_size': 10, },
    'test_ratio':.3,
    'test_metric': MetricTypes.MEAN_SQUARE_ERROR,
    'test_on_local_updates': True,
    'test_on_global_updates': True
}

In [ ]:
from fedbiomed.researcher.federated_workflows import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['adni']

# Add more rounds for results with better accuracy
#
#rounds = 40
rounds = 2

# select nodes participating to this experiment
exp = Experiment(tags=tags,
                 model_args=model_args,
                 training_plan_class=SGDRegressorTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

In [ ]:
# start federated training
exp.run()

Save trained model to file

In [ ]:
exp.training_plan().export_model('./trained_model')

In [ ]:
exp.aggregated_params()

In [ ]:
fed_model = exp.training_plan().model()
fed_model.intercept_ = exp.aggregated_params()[rounds-1]['params']['intercept_']
fed_model.coef_ = exp.aggregated_params()[rounds-1]['params']['coef_']

## SGDClassifier
### Purpose of the exercise :

Three datasets `c1_3class.csv` , `c2_3class.csv` and `c3_3class.csv` has been generated with a target column of 3 different classes.
We will fit a SGCClassifier (classifier) using Federated Learning.

### Get the data 

We use the make_classification dataset from sklearn datasets

In [ ]:
from sklearn import datasets
import numpy as np

In [ ]:
X,y = datasets.make_classification(n_samples=300, n_features=20,n_informative = 3, n_classes=3,n_clusters_per_class=2, weights=None, flip_y=0.01, class_sep=1.0, hypercube=True, shift=0.0, scale=1.0,shuffle=True, random_state=123)
X.shape,y.shape

In [ ]:
C1 = X[:150,:]
C2 = X[150:250,:]
C3 = X[250:300,:]

y1 = y[:150].reshape([150,1])
y2 = y[150:250].reshape([100,1])
y3 = y[250:300].reshape([50,1])

C1.shape ,C2.shape , C3.shape , y1.shape, y2.shape, y3.shape

In [ ]:
n1 = np.concatenate((C1, y1), axis=1)
np.savetxt('./data/c1_3class.csv',n1,delimiter=',')

n2 = np.concatenate((C2, y2), axis=1)
np.savetxt('./data/c2_3class.csv',n2,delimiter=',')

n3 = np.concatenate((C3, y3), axis=1)
np.savetxt('./data/c3_3class.csv',n3,delimiter=',')

### Setting the node up
Before running this notebook you need to configure 2 nodes: <br/>
* **Node 1 :** `fedbiomed node  -d my-first-node dataset add`
  * Select option 1 to add a csv file to the node
  * Choose the name, tags and description of the dataset (you can write **'perp1'** always and it will be good)
  * Pick the c1_3class.csv file in your machine  (in `notebooks/data/c1_3class.csv`)
  * Check that your data has been added in node 1 by executing `fedbiomed node -d my-first-node dataset list`
  * Run the node using `fedbiomed node -d my-first-node start`. <br/>

* **Node 2 :** Open a second terminal and run `fedbiomed node -d my-second-node dataset add` 
  * Select option 1 to add a csv file to the node
  * Choose the name, tags and description of the dataset (you can write **'perp1'** always and it will be good)
  * Pick the c2_3class.csv file in your machine (in `notebooks/data/c2_3class.csv`)
  * Check that your data has been added in node 2 by executing `fedbiomed node -d my-second-node dataset list`
  * Run the node using `fedbiomed node -d my-second-node start`.
 

 Wait until you get `Starting task manager`. it means node is online.


**model_args** is a dictionnary containing your model arguments, in case of SGDRegressor this will be max_iter and tol.

**training_args** is a dictionnary with parameters , related to Federated Learning. 

In [ ]:
n_features = 20
n_classes = 3

model_args = {'max_iter':1000, 'tol': 1e-3 , 
               'n_features' : n_features, 'n_classes' : n_classes}

training_args = {   
    'epochs': 5,
    'loader_args': { 'batch_size': 1, },
}

Hereafter the template of the class you should provide to Fedbiomed :
    
**training_data** : you must return here the (X,y) that must be of the same type of 
your method partial_fit parameters. 

In [ ]:
from fedbiomed.common.training_plans import FedSGDClassifier
from fedbiomed.common.data import DataManager


class SGDClassifierTrainingPlan(FedSGDClassifier):
    def training_data(self):
        NUMBER_COLS = 20
        dataset = pd.read_csv(self.dataset_path,header=None,delimiter=',')
        X = dataset.iloc[:,0:NUMBER_COLS].values
        y = dataset.iloc[:,NUMBER_COLS]       
        return DataManager(dataset=X,target=y.values, shuffle=True)

In [ ]:
from fedbiomed.researcher.federated_workflows import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['perp1']
rounds = 2

# search for corresponding datasets across nodes datasets
exp = Experiment(tags=tags,
                 model_args=model_args,
                 training_plan_class=SGDClassifierTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)


In [ ]:
exp.run()

Save trained model to file

In [ ]:
exp.training_plan().export_model('./trained_model')

In [ ]:
import pandas as pd
data = pd.read_csv('./data/c3_3class.csv')

In [ ]:
X_test = data.iloc[:,:n_features]
y_test = data.iloc[:,n_features]

F1 score computed with federated algorithm :

For that, we are exporting `exp.aggregated_params()` containing models parameters collected at the end of each round

In [ ]:
from sklearn.metrics import classification_report, f1_score
loss_metric = f1_score
    
testing_error = []

for i in range(rounds):
    fed_model = exp.training_plan().model()
    fed_model.coef_ = exp.aggregated_params()[i]['params']['coef_']
    fed_model.intercept_ = exp.aggregated_params()[i]['params']['intercept_']
    print(f'Model trained in round {i}')
    print('-------------------------')
    print(classification_report(y_test, fed_model.predict(X_test), digits=3))